# 🛠️ Einführung: Function Calling mit vLLM

In dem Notebook lernen wir, wie man LLMs dazu bringt, Funktionen aufzurufen und Tool-Interaktionen durchzuführen.  

## 📚 Quellen

- [vLLM: Tool Calling Documentation](https://docs.vllm.ai/en/latest/features/tool_calling.html)
- [OpenAI: Function Calling API](https://platform.openai.com/docs/guides/function-calling)

---

Viel Erfolg beim Ausprobieren der Function Calling Features! 🤗

In [41]:
%%capture
!pip install ollama

In [42]:
LLM_URL = "http://132.199.138.16:11434"
LLM_MODEL = "gpt-oss:20b" # Nicht jedes LLM unterstützt Function Calling. Gemma3 beispielsweise wurde nicht dafür trainiert. Das OpenSource LLM "gpt-oss:20b" von OpenAI hingegen schon.

In [43]:
from ollama import Client

# Verwenden wir zur Abwechslung mal die ollama Bibliothek statt openai. Tool Calling wird auch von openai unterstützt.
# Siehe hier: https://ollama.com/blog/tool-support
client = Client(
  host=LLM_URL
)

### 1. Beispiel: Tool Call mit Aktienkursen

Definieren wir zunächst eine Liste mit Tools. In unserem Beispiel nur **ein** Tool, das den aktuellen Aktienkurs für ein gegebenes Symbol zurückgibt. 
Die `openai`-Bibliothek erwartet jedes Tool in einem bestimmten JSON-Format. 

* `type`: OpenAI bietet prinzipiell auch andere Tool-Typen an (z.B. `web_search`), wir verwenden hier aber nur `function`.
* `name`: Der Name des Tools.
* `parameters`: Die Parameter, die die Funktion erwartet.
* `required`: Welche Parameter zwingend übergeben werden müssen.

In [92]:
tools = [{
    'type': 'function',
    'function': {
            'name': 'get_stock_price',
            'description': 'Get the current stock price for a company',
            'parameters': {
                'type': 'object',
                'properties': {
                    'symbol': {
                        'type': 'string',
                    },
                },
                'required': ['symbol'],
            },
    },
}]

In [93]:
# Starten wir mit einem Beispiel, in dem wir den aktuellen Aktienkurs von Apple abfragen.
response = client.chat(
    model=LLM_MODEL,
    messages=[{'role': 'user', 'content': 'Wie ist die aktuelle Apple Aktie?'}],
    tools=tools,
)

tool_calls = response['message']['tool_calls']
print(f"Tool calls: {tool_calls}")
# Argumente des Tool Calls und Funktion ausgeben
print(f"Function name: {tool_calls[0].function.name}")
print(f"Function arguments: {tool_calls[0].function.arguments}")

Tool calls: [ToolCall(function=Function(name='get_stock_price', arguments={'symbol': 'AAPL'}))]
Function name: get_stock_price
Function arguments: {'symbol': 'AAPL'}


In [94]:
# Fragen wir eine irrelevante Frage, die nichts mit Aktien zu tun hat.
# In dem Fall ist tool_calls None, da das LLM kein Tool aufrufen muss.
response = client.chat(
    model=LLM_MODEL,
    messages=[{'role': 'user', 'content': 'Was ist 1+1?'}],
    tools=tools,
)

# Bei der Ausgabe sehen wir, dass tool_calls None ist.
print(response['message'])

role='assistant' content='1\u202f+\u202f1\u202f=\u202f2.' thinking='User asks in German: "Was ist 1+1?" meaning "What is 1+1?" The answer: 2.' images=None tool_name=None tool_calls=None


## Übungsaufgabe: Tool Execution - Funktionen tatsächlich ausführen

Unser Wetter Tool können wir nun tatsächlich implementieren mit einer echten API (siehe Beispiel-Aufruf unten). Diese stellt Wetterdaten bereit für heute und morgen.

Gehen Sie für die Implementierung wie folgt vor:

1. Definieren Sie eine Funktion `get_weather` im JSON Schema
2. Implementieren Sie die Funktion `get_weather_from_api`, die die Wetter API aufruft und die Wetterdaten für einen bestimmten Ort abruft. Die Funktion gibt alle Wetter-Daten als Python Dictionary zurück.
3. Rufen Sie das LLM auf mit `get_weather` als verfügbares Tool
4. Analysieren Sie die Antwort des LLM und extrahieren Sie die Parameter für den Funktionsaufruf
5. Rufen Sie die Funktion `get_weather_from_api` mit den extrahierten Parametern auf
6. Geben Sie die Ergebnisse der Funktion zurück an das LLM, um eine abschließende Antwort zu generieren


So funktioniert die API:

In [9]:
import requests

city = "Berlin"
url = f"http://wttr.in/{city}?format=j1"  # JSON-Format
response = requests.get(url)
data = response.json()

print("Ort:", city)
print("Aktuelle Temperatur:", data["current_condition"][0]["temp_C"], "°C")
print("Wetter:", data["current_condition"][0]["weatherDesc"][0]["value"])

# Zugriff auf Wetterdaten
# data["weather"][0] # Wetter heute
# data["weather"][1] # Wetter morgen
print([tempData["time"] for tempData in data["weather"][0]["hourly"]]) # Output: ['0', '300', '600', '900', '1200', '1500', '1800', '2100'], was die Stunden des Tages repräsentiert
print([tempData["tempC"] for tempData in data["weather"][0]["hourly"]]) # Temperaturen heute für 0 bis 21 Uhr in 3-Stunden-Schritten


Ort: Berlin
Aktuelle Temperatur: 12 °C
Wetter: Light drizzle
['0', '300', '600', '900', '1200', '1500', '1800', '2100']
['13', '13', '11', '12', '15', '14', '13', '13']


<details>
<summary><b>Lösung anzeigen</b></summary>

```python
import requests
import json

# 1. JSON Schema Definition für get_weather Tool
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'get_weather',
            'description': 'Ruft aktuelle Wetterdaten für eine Stadt ab',
            'parameters': {
                'type': 'object',
                'properties': {
                    'city': {
                        'type': 'string',
                        'description': 'Name der Stadt'
                    }
                },
                'required': ['city']
            }
        }
    }
]

# 2. API-Funktion implementieren


def get_weather_from_api(city):
    """Ruft Wetterdaten von wttr.in API ab"""
    url = f"http://wttr.in/{city}?format=j1"
    try:
        response = requests.get(url)
        data = response.json()

        # Strukturierte Rückgabe
        weather_info = {
            "city": city,
            "current_temp": data["current_condition"][0]["temp_C"],
            "current_weather": data["current_condition"][0]["weatherDesc"][0]["value"],
            "today_temps": [temp["tempC"] for temp in data["weather"][0]["hourly"]],
            "today_times": [temp["time"] for temp in data["weather"][0]["hourly"]],
            "tomorrow_temps": [temp["tempC"] for temp in data["weather"][1]["hourly"]],
            "tomorrow_times": [temp["time"] for temp in data["weather"][1]["hourly"]],
            "tomorrow_temp": data["weather"][1]["avgtempC"],
        }
        return weather_info
    except Exception as e:
        return {"error": f"Fehler beim Abrufen der Wetterdaten: {str(e)}"}


# 3. Vollständiger Workflow
def weather_tool_workflow(user_question):

    # Führen wir zunächst den Tool-Call aus.
    messages_tool_call = [{'role': 'user', 'content': user_question}]
    response = client.chat(
        model=LLM_MODEL,
        messages=messages_tool_call,
        tools=tools,
    )
    tool_calls = response['message']['tool_calls']

    # Prüfe, ob das LLM ein Tool aufrufen würde
    if tool_calls:
        
        # Speichern wir function_name und arguments in eine Variable
        tool_call = tool_calls[0]
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']

        print(f"Tool aufgerufen: {function_name}")
        print(f"Parameter: {arguments}")

        # Rufen wir nun die "echte" get_wheather_from_api auf
        if function_name == "get_weather":
            weather_data = get_weather_from_api(arguments["city"])
            print("Abgerufene Wetterdaten:", weather_data)

        # Die ursprüngliche User Prompt sowie das Ergebnis vom Tool-Aufruf geben wir nun an das LLM um die
        # Finale Antwort zu generieren. 
        # json.dumps wandelt das Ergebnis der Wetter API von einem Python Dictionary um in ein String. Das LLM unterstützt als Input immer nur Text.
        messages = [{'role': 'user', 'content': f"Der User hat folgende frage gestellt {user_question}. Bitte antworte in einem kleinen Fließtext auf die Frage."}]
        messages.append({'role': 'tool', 'content': json.dumps(weather_data)})

        # Finale Antwort vom LLM
        final_response = client.chat(
            model=LLM_MODEL,
            messages=messages,
            tools=tools,
        )

        return final_response['message']['content']

    return response['message']['content']


result = weather_tool_workflow(
    "Wie ist das Wetter morgen am Abend in München?")
print("Finale Antwort:", result)
```

</details>